In [108]:
from melanet import MelaData

import os
import random
import numpy as np
import shutil

import torch
import torch.nn as nn
import torch.nn.functional as F

# 0 - Pre-processing on images

In [44]:
%mkdir data
%cd data/

/Users/robinali/Documents/GitHub/Skin_Cancer_Detection/data


In [45]:
!pip install kaggle

You are using pip version 9.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [66]:
!kaggle datasets download -d kmader/skin-cancer-mnist-ham10000

100%|█████████████████████████████████████▉| 2.62G/2.62G [09:54<00:00, 4.63MB/s]
100%|██████████████████████████████████████| 2.62G/2.62G [09:54<00:00, 4.73MB/s]


In [67]:
!unzip skin-cancer-mnist-ham10000.zip

Archive:  skin-cancer-mnist-ham10000.zip
  inflating: hmnist_28_28_RGB.csv    
  inflating: HAM10000_metadata.csv   
  inflating: HAM10000_images_part_1.zip  
  inflating: hmnist_28_28_L.csv      
  inflating: hmnist_8_8_L.csv        
  inflating: HAM10000_images_part_2.zip  
  inflating: hmnist_8_8_RGB.csv      


In [68]:
%rm skin-cancer-mnist-ham10000.zip

In [83]:
!chmod 755 *.zip

In [112]:
%cd images

/Users/robinali/Documents/GitHub/Skin_Cancer_Detection/data/images
mkdir: test: File exists
mkdir: train: File exists


In [113]:
test_prop = 0.1
test_files = random.sample(os.listdir(), int(test_prop*len(os.listdir())))

In [ ]:
%mkdir test
%mkdir train

In [125]:
os.listdir().remove('train')

In [126]:
for f in test_files:
    shutil.move(f, 'test')

Error: Destination path 'test/ISIC_0026266.jpg' already exists

In [128]:
for f in [x for x in os.listdir() if x not in ['test', 'train']]:
    shutil.move(f, 'train')

# 1 - Load the data

In [25]:
base_dir = '/Users/robinali/Desktop/HEC/2018 - 2019/project/skin-cancer-mnist-ham10000/'

In [37]:
train = MelaData(data_dir=base_dir+'train/', label_csv=base_dir+'HAM10000_metadata.csv')
test = MelaData(data_dir=base_dir+'test/', label_csv=base_dir+'HAM10000_metadata.csv')

In [40]:
bs = 64
train_loader = torch.utils.data.DataLoader(train, batch_size=bs, shuffle=True)
test_loader = torch.utils.data.DataLoader(test, batch_size=bs, shuffle=False)

## 2 - Basic Neural Network

A basic NN to predict melanomes from raw data

In [41]:
class classifier(nn.Module):
    
    def __init__(self):
        super(classifier, self).__init__()
        # fill the missing entries below
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3, padding=1)
        self.fc = nn.Linear(in_features=128, out_features=2)
        
    def forward(self,x):
        # implement your network here, use F.max_pool2d, F.log_softmax and do not forget to flatten your vector
        x = self.conv1(x)
        x = F.max_pool2d(x,7)
        x = x.view(x.size(0),-1)
        x = self.fc(x)
        x = F.log_softmax(x)
        return x

In [13]:
conv_class = classifier()

In [14]:
use_gpu = torch.cuda.is_available()
if use_gpu:
    conv_class = conv_class.cuda()